In [1]:
import os
from glob import glob # extract path of each file
import pandas as pd # data preprocessing
from functools import reduce
from xml.etree import ElementTree as et # parse information from XML

In [2]:
# Load all xml files and store in a list
# step-1: get path of each xml file
xmlfiles = glob('./data_images/*.xml')
# data cleaning. replace \\ with /
replace_text = lambda x: x.replace('\\','/')
xmlfiles = list(map(replace_text,xmlfiles))

In [3]:
xmlfiles

['./data_images/000001.xml',
 './data_images/000002.xml',
 './data_images/000007.xml',
 './data_images/000009.xml',
 './data_images/000012.xml',
 './data_images/000016.xml',
 './data_images/000017.xml',
 './data_images/000019.xml',
 './data_images/000020.xml',
 './data_images/000021.xml',
 './data_images/000023.xml',
 './data_images/000024.xml',
 './data_images/000026.xml',
 './data_images/000030.xml',
 './data_images/000032.xml',
 './data_images/000033.xml',
 './data_images/000034.xml',
 './data_images/000035.xml',
 './data_images/000036.xml',
 './data_images/000039.xml',
 './data_images/000041.xml',
 './data_images/000042.xml',
 './data_images/000044.xml',
 './data_images/000046.xml',
 './data_images/000047.xml',
 './data_images/000048.xml',
 './data_images/000050.xml',
 './data_images/000051.xml',
 './data_images/000052.xml',
 './data_images/000060.xml',
 './data_images/000061.xml',
 './data_images/000063.xml',
 './data_images/000064.xml',
 './data_images/000065.xml',
 './data_image

In [4]:
# step-2: read xml files
# from each xml file we need to extract
# filename, size(with, height), object(name, xmin, xmax, ymin, ymax)
def extract_text(filename):
    tree = et.parse(filename)
    root = tree.getroot()
    
    # extract filename
    image_name = root.find('filename').text
    # width and height of the image
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name,width,height,name,xmin,xmax,ymin,ymax])
    
    return parser

In [5]:
parser_all = list(map(extract_text, xmlfiles))

In [6]:
data = reduce(lambda x, y: x + y, parser_all)

In [7]:
df = pd.DataFrame(data, columns = ['filename', 'width', 'height', 'name', 'xmin', 'xmax', 'ymin', 'ymax'])

In [8]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,000001.jpg,1024,657,car,14,301,335,522
1,000001.jpg,1024,657,car,269,571,345,489
2,000001.jpg,1024,657,car,502,798,342,450
3,000001.jpg,1024,657,car,709,1009,333,438
4,000002.jpg,800,600,car,41,768,240,497


In [9]:
df.shape

(1572, 8)

In [10]:
df['name'].value_counts()

name
person         580
car            187
chair          167
bicycle         51
boat            48
dog             47
bottle          44
bird            44
horse           43
aeroplane       42
sofa            41
train           38
pottedplant     37
cat             34
tvmonitor       31
cow             30
diningtable     29
motorbike       28
sheep           26
bus             25
Name: count, dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1572 entries, 0 to 1571
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  1572 non-null   object
 1   width     1572 non-null   object
 2   height    1572 non-null   object
 3   name      1572 non-null   object
 4   xmin      1572 non-null   object
 5   xmax      1572 non-null   object
 6   ymin      1572 non-null   object
 7   ymax      1572 non-null   object
dtypes: object(8)
memory usage: 98.4+ KB


In [12]:
# type conversion
cols = ['width', 'height', 'xmin', 'xmax', 'ymin', 'ymax']
df[cols] = df[cols].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1572 entries, 0 to 1571
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  1572 non-null   object
 1   width     1572 non-null   int64 
 2   height    1572 non-null   int64 
 3   name      1572 non-null   object
 4   xmin      1572 non-null   int64 
 5   xmax      1572 non-null   int64 
 6   ymin      1572 non-null   int64 
 7   ymax      1572 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 98.4+ KB


In [13]:
# center x, center y
df['center_x'] = ((df['xmax'] + df['xmin']) / 2) / df['width']
df['center_y'] = ((df['ymax'] + df['ymin']) / 2) / df['height']
# w
df['w'] = (df['xmax'] - df['xmin']) / df['width']
# h
df['h'] = (df['ymax'] - df['ymin']) / df['height']

In [14]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,000001.jpg,1024,657,car,14,301,335,522,0.153809,0.652207,0.280273,0.284627
1,000001.jpg,1024,657,car,269,571,345,489,0.410156,0.634703,0.294922,0.219178
2,000001.jpg,1024,657,car,502,798,342,450,0.634766,0.602740,0.289062,0.164384
3,000001.jpg,1024,657,car,709,1009,333,438,0.838867,0.586758,0.292969,0.159817
4,000002.jpg,800,600,car,41,768,240,497,0.505625,0.614167,0.908750,0.428333


## split data into train and test

In [15]:
images = df['filename'].unique()

In [16]:
len(images)

500

In [17]:
# 80% train and 20% test
img_df = pd.DataFrame(images, columns=['filename'])
img_train = tuple(img_df.sample(frac=0.8)['filename']) # shuffle and pick 80% of images

In [18]:
img_test = tuple(img_df.query(f'filename not in {img_train}')['filename']) # take rest 20% images

In [19]:
len(img_train), len(img_test)

(400, 100)

In [20]:
train_df = df.query(f'filename in {img_train}')
test_df = df.query(f'filename in {img_test}')

In [21]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,000001.jpg,1024,657,car,14,301,335,522,0.153809,0.652207,0.280273,0.284627
1,000001.jpg,1024,657,car,269,571,345,489,0.410156,0.634703,0.294922,0.219178
2,000001.jpg,1024,657,car,502,798,342,450,0.634766,0.602740,0.289062,0.164384
3,000001.jpg,1024,657,car,709,1009,333,438,0.838867,0.586758,0.292969,0.159817
4,000002.jpg,800,600,car,41,768,240,497,0.505625,0.614167,0.908750,0.428333


In [22]:
test_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
6,000007.jpg,500,333,car,141,500,50,330,0.641000,0.570571,0.718000,0.840841
11,000012.jpg,500,333,car,156,351,97,270,0.507000,0.551051,0.390000,0.519520
18,000021.jpg,336,500,dog,1,182,235,388,0.272321,0.623000,0.538690,0.306000
19,000021.jpg,336,500,person,210,336,36,482,0.812500,0.518000,0.375000,0.892000
20,000021.jpg,336,500,person,46,170,82,365,0.321429,0.447000,0.369048,0.566000


## Assign id number to object names

In [25]:
# label encoding
def label_encoding(x):
    labels = {'person':0, 'car':1, 'chair':2, 'bottle':3, 'pottedplant':4, 'bird':5, 'dog':6,
             'sofa':7, 'bicycle':8, 'horse':9, 'boat':10, 'motorbike':11, 'cat':12, 'tvmonitor':13,
             'cow':14, 'sheep':15, 'aeroplane':16, 'train':17, 'diningtable':18, 'bus':19}
    return labels[x]

In [26]:
train_df['id'] = train_df['name'].apply(label_encoding)
test_df['id'] = test_df['name'].apply(label_encoding)

C:\Users\LeVietTien\AppData\Local\Temp\ipykernel_8912\3116627190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['id'] = train_df['name'].apply(label_encoding)
C:\Users\LeVietTien\AppData\Local\Temp\ipykernel_8912\3116627190.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['id'] = test_df['name'].apply(label_encoding)


In [27]:
train_df.head(10)

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,id
0,000001.jpg,1024,657,car,14,301,335,522,0.153809,0.652207,0.280273,0.284627,1
1,000001.jpg,1024,657,car,269,571,345,489,0.410156,0.634703,0.294922,0.219178,1
2,000001.jpg,1024,657,car,502,798,342,450,0.634766,0.602740,0.289062,0.164384,1
3,000001.jpg,1024,657,car,709,1009,333,438,0.838867,0.586758,0.292969,0.159817,1
4,000002.jpg,800,600,car,41,768,240,497,0.505625,0.614167,0.908750,0.428333,1
5,000002.jpg,800,600,car,533,722,236,299,0.784375,0.445833,0.236250,0.105000,1
7,000009.jpg,500,375,horse,69,270,172,330,0.339000,0.669333,0.402000,0.421333,9
8,000009.jpg,500,375,person,150,229,141,284,0.379000,0.566667,0.158000,0.381333,0
9,000009.jpg,500,375,person,285,327,201,331,0.612000,0.709333,0.084000,0.346667,0
10,000009.jpg,500,375,person,258,297,198,329,0.555000,0.702667,0.078000,0.349333,0


## Save Image and Labels in text

In [28]:
from shutil import move

In [29]:
train_folder = 'data_images/train'
test_folder = 'data_images/test'

os.mkdir(train_folder)
os.mkdir(test_folder)

In [30]:
cols = ['filename', 'id', 'center_x', 'center_y', 'w', 'h']
groupby_obj_train = train_df[cols].groupby('filename')
groupby_obj_test = test_df[cols].groupby('filename')

In [35]:
# groupby_obj_train.get_group('000001.jpg').set_index('filename').to_csv('sample.txt', index=False, header=False)
# save each image in train/test folder and repective labels in .txt
def save_data(filename, folder_path, group_obj):
    # move image
    src = os.path.join('data_images', filename)
    dest = os.path.join(folder_path, filename)
    move(src, dest) # move image to the destination folder

    # save the labels
    text_filename = os.path.join(folder_path, os.path.splitext(filename)[0] + '.txt')
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename, sep=' ', index=False, header=False)

In [38]:
filename_series = pd.Series(groupby_obj_train.groups.keys())

In [43]:
filename_series.apply(save_data, args=(train_folder, groupby_obj_train))

0      None
1      None
2      None
3      None
4      None
       ... 
395    None
396    None
397    None
398    None
399    None
Length: 400, dtype: object

In [44]:
filename_series_test = pd.Series(groupby_obj_test.groups.keys())
filename_series_test.apply(save_data, args=(test_folder, groupby_obj_test))

0     None
1     None
2     None
3     None
4     None
      ... 
95    None
96    None
97    None
98    None
99    None
Length: 100, dtype: object